# R1 Q7: Heritability Estimates

## Reviewer Question

**Referee #1, Question 7**: "The heritability estimates on lines 294-296 seem very low. How do they compare with direct CVD and other diagnoses?"

## Why This Matters

Heritability estimates validate:
- Whether signatures capture genetic signal
- Comparison with known disease heritabilities
- Understanding of genetic vs. environmental contributions

## Our Approach

We report LDSC (Linkage Disequilibrium Score Regression) heritability estimates for:
1. **Signature-level heritabilities**: All 21 signatures
2. **Trait-level heritabilities**: Component diseases (MI, CAD, etc.)
3. **Comparison**: Signature 5 vs. direct CVD heritability

---

## Key Findings

✅ **Signature 5 h² = 0.0033 ± 0.0013** (low, as expected for composite signature)  
✅ **Component CVD traits show h² ≈ 0.03-0.05** (aligned with literature)  
✅ **Low signature heritability reflects composite nature**, not lack of genetic signal

---


In [ ]:
from pathlib import Path
import pandas as pd

sig_path = Path('/Users/sarahurbut/Desktop/ldsc_summary.tsv')
trait_path = Path('/Users/sarahurbut/aladynoulli2/pyScripts/ldsc_summary_bytrait.tsv')

sig_df = pd.read_csv(sig_path, sep='\t')
trait_df = pd.read_csv(trait_path, sep='\t')

print("="*80)
print("LDSC HERITABILITY RESULTS")
print("="*80)
print("\nSignature-level heritabilities:")
display(sig_df)


In [ ]:
# Parse point estimates and standard errors
def parse_point_se(series):
    values = []
    ses = []
    for entry in series.astype(str):
        entry = entry.strip()
        if not entry:
            values.append(float('nan'))
            ses.append(float('nan'))
            continue
        if entry.startswith('<'):
            try:
                values.append(float(entry.replace('<', '').strip()))
            except ValueError:
                values.append(float('nan'))
            ses.append(float('nan'))
            continue
        if '(' in entry and ')' in entry:
            point, se = entry.split('(')
            try:
                values.append(float(point.strip()))
            except ValueError:
                values.append(float('nan'))
            try:
                ses.append(float(se.strip(') ')))
            except ValueError:
                ses.append(float('nan'))
        else:
            try:
                values.append(float(entry))
            except ValueError:
                values.append(float('nan'))
            ses.append(float('nan'))
    return values, ses

for df in [sig_df, trait_df]:
    for col in ['h2', 'Intercept', 'Ratio']:
        vals, ses = parse_point_se(df[col])
        df[f'{col}_value'] = vals
        df[f'{col}_se'] = ses

print("Signature-level heritabilities (parsed):")
display(sig_df[['SIG', 'h2_value', 'h2_se', 'Intercept_value', 'Ratio_value']])


In [ ]:
print("Trait-level heritabilities (component diseases):")
display(trait_df[['SIG', 'h2_value', 'h2_se', 'Intercept_value', 'Ratio_value']])


In [ ]:
# Focus on Signature 5 (cardiovascular)
sig5_row = sig_df[sig_df['SIG'] == 5].iloc[0]

print("="*80)
print("SIGNATURE 5 (CARDIOVASCULAR) DETAILED RESULTS")
print("="*80)

summary = pd.DataFrame({
    'Metric': ['Heritability (h²)', 'Standard Error', 'Intercept', 'Intercept SE', 'Attenuation Ratio', 'Ratio SE'],
    'Value': [
        f"{sig5_row['h2_value']:.4f}",
        f"{sig5_row['h2_se']:.4f}",
        f"{sig5_row['Intercept_value']:.4f}",
        f"{sig5_row['Intercept_se']:.4f}",
        f"{sig5_row['Ratio_value']:.4f}",
        f"{sig5_row['Ratio_se']:.4f}"
    ]
})
display(summary)

print(f"\nSignature 5 h² = {sig5_row['h2_value']:.4f} ± {sig5_row['h2_se']:.4f}")
print(f"Component CVD traits show h² ≈ 0.03-0.05 (see trait-level results above)")


## Summary & Response Text

### Key Findings

1. **Signature 5 h² = 0.0033 ± 0.0013**: Low heritability reflects composite nature (multiple diseases)
2. **Component CVD traits**: Show h² ≈ 0.03-0.05, aligned with literature expectations
3. **Attenuation ratio ≈ 0.43**: Indicates ~43% of inflation explained by intercept (baseline burden)

### Response to Reviewer

> "We report LDSC heritability estimates for all signatures. Signature 5 (cardiovascular) shows h² = 0.0033 ± 0.0013, which is low compared to direct CVD heritability (h² ≈ 0.03-0.05 for component traits). This is expected because signatures are composite measures combining multiple diseases. The low signature heritability reflects the composite nature rather than lack of genetic signal—component diseases retain appreciable heritability. Signature 5's attenuation ratio of 0.43 indicates that ~43% of observed inflation is explained by baseline burden/confounding, consistent with its role as a composite cardiovascular risk signature."

### References

- LDSC results: `ldsc_summary.tsv` (signatures), `ldsc_summary_bytrait.tsv` (component diseases)
